# Signal Classifier

## Library Imports

In [1]:
import numpy as np
import plotly.graph_objects as go
import tensorflow as tf
#np.seterr(divide='ignore', invalid='ignore')
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
def plot(fig,x,y,name):
    fig.add_trace(go.Scatter(x=x,y=y,mode='lines',name=name))


In [2]:
!pip install tensorflow

     |████████████████████████████████| 462 kB 5.6 MB/s 


## Defintion of Different Signal Functions

In [3]:

def box(x,a=2,b=5,noise=False,noiseParameter=0.1):
    y=np.zeros(len(x))
    y[np.abs(x)<a]=b
    if noise:
      y = y + np.random.normal(0,noiseParameter,len(x))
    return y
def saw(x,a=2,b=5,noise=False,noiseParameter=0.1):
    y=np.zeros(len(x))
    y[np.abs(x)<a]=-b*x[np.abs(x)<a]
    if noise:
      y = y + np.random.normal(0,noiseParameter,len(x))    
      
    return y
def exp(x,a=2,b=2,c=-1,noise=False,noiseParameter=0.1):
    y=np.zeros(len(x))
    y[np.abs(x)<a]=c+np.e**(-b*x[[np.abs(x)<a]])
    if noise:
      y = y + np.random.normal(0,noiseParameter,len(x))    
    return y
def sinu(x,a=2,b=2,c=-1,noise=False,noiseParameter=0.1):
    y=np.zeros(len(x))
    y[np.abs(x)<a]=c*np.sin(b*x[np.abs(x)<a])
    if noise:
      y = y + np.random.normal(0,noiseParameter,len(x))    
    return y
def gaus(x,a=2,b=2,noise=False,noiseParameter=0.1):
    y=np.zeros(len(x))
    #y[np.abs(x)<a]=np.e**(-a*(x[np.abs(x)<a]**(2*b)))
    y=np.e**(-a*(x**(2*b)))
    if noise:
      y = y + np.random.normal(0,noiseParameter,len(x))    
    return np.nan_to_num(y)
x=np.linspace(-np.pi,np.pi,100)

## Generating 10000 Functions of each signal for Training Model

### Box Function

In [4]:
a=np.linspace(np.pi/4,np.pi/2,10000)
b=np.append(np.linspace(-1,-0.01,5000),np.linspace(0.01,1,5000))
a=np.random.permutation(a)
b=np.random.permutation(b)
BoxSignal=[]
for i in range(10000):
    F=box(x,a[i],b[i])
    normalization=np.max(abs(F))
    BoxSignal.append(F/normalization)


### Saw Function

In [5]:
a=np.linspace(np.pi/4,np.pi/2,10000)
b=np.append(np.linspace(-1,-0.01,5000),np.linspace(0.01,1,5000))
a=np.random.permutation(a)
b=np.random.permutation(b)
SawSignal=[]
#normalization=np.max(abs(saw(x,max(a),max(b))))
for i in range(10000):
    F=saw(x,a[i],b[i])
    normalization=np.max(abs(F))
    SawSignal.append(F/normalization)

### Exponential

In [6]:
a=np.linspace(np.pi/4,np.pi/2,10000)
b=np.append(np.linspace(-1,-0.01,5000),np.linspace(0.01,1,5000))
c=np.append(np.linspace(-3,-1.01,5000),np.linspace(-1.01,1,5000))
a=np.random.permutation(a)
b=np.random.permutation(b)
c=np.random.permutation(c)
ExpSignal=[]
for i in range(10000):
    F=exp(x,a[i],b[i],c[i])
    normalization=np.max(abs(F))
    ExpSignal.append(F/normalization)

### Sinusoid

In [7]:
a=np.linspace(np.pi/4,np.pi/2,10000)
b=np.append(np.linspace(-1,-0.01,5000),np.linspace(0.01,2*np.pi,5000))
c=np.append(np.linspace(-1,-0.01,5000),np.linspace(0.01,1,5000))
a=np.random.permutation(a)
b=np.random.permutation(b)
c=np.random.permutation(c)
SinSignal=[]
for i in range(10000):
    F=sinu(x,a[i],b[i],c[i])
    normalization=np.max(abs(F))
    SinSignal.append(sinu(x,a[i],b[i],c[i])/normalization)

### Gaussian

In [8]:
#a=np.linspace(0.01,3,10000)
a=np.linspace(np.pi/4,np.pi/2,10000)
b=np.linspace(-10,10,10000)
a=np.random.permutation(a)
b=np.random.permutation(b)

GausSignal=[]
for i in range(10000):
    F=gaus(x,a[i],int(b[i]))
    normalization=np.max(abs(F))
    GausSignal.append(F/normalization)

### Combining Results

In [9]:
Signals=[BoxSignal,SawSignal,ExpSignal,SinSignal,GausSignal]
SignalLabels=[
    np.zeros(10000),
    1*np.ones(10000),
    2*np.ones(10000),
    3*np.ones(10000),
    4*np.ones(10000)]
SignalsTrain=np.array(np.concatenate(Signals).flat)
SignalsTrain=SignalsTrain.reshape((50000,100))
SignalLabelsTrain=np.array(np.concatenate(SignalLabels).flat)

## Generating 1000 Functions + Fourier Series Representation of each signal for Testing Model

### Box Function

In [10]:
## Generating 1000 functions of each for testing model
#Box
a=np.linspace(np.pi/4,np.pi/2,1000)
b=np.append(np.linspace(-1,-0.01,500),np.linspace(0.01,1,500))
a=np.random.permutation(a)
b=np.random.permutation(b)
BoxSignal=[]
for i in range(1000):
    F=box(x,a[i],b[i])
    normalization=np.max(abs(F))
    BoxSignal.append(F/normalization)






#### Fourier Representation


In [11]:

def e(n,x,positve=True):
    if(positve):
        return np.e**(1j*n*x)
    else:
        return np.e**(-1j*n*x)
''' TOO SLOW for COLAB
def calculateCn(M1,D,F):
    for p in range(M1-1):
        for q in range(M1-1):
            D[p][q]=(e(n[p],xi[q],False))
    cn=np.dot(D,F)/M1
    return cn
  ##Fourier Series Representation

  M1=200 #number of grid points
  M2 = 1500
  h=np.linspace(0,M1-1,M1-1) #
  xi=-np.pi+(2*np.pi/M1)*h
  D=np.zeros((M1-1,M1-1),dtype=complex)
  n=np.linspace(-M1/2,M1/2,M1-1)
  F=box(xi,a[i],b[i])
  cn=calculateCn(M1,D,F)
  h2=np.linspace(0,M2-1,M2-1)
  xi2 =-np.pi+ (2*np.pi/M2)*h2
  xi=np.linspace(-np.pi,np.pi,100)
  D2=np.zeros((M2-1,M1-1),dtype=complex)
  for p in range(M2-1):
    for q in range(M1-1):
      D2[p][q]=(e(xi2[p],n[q]))

  fx2 = (np.dot(D2,cn)).real
  FourierBox.append(fx2)

#TODO
#figure out fft


fx=[]
for xi in x:
    result=0
    for n in range(int((-N/2)),int((N/2))):
      result=result+(cn[int(i+(N/2))]*(np.e**(1j*n*xi))) #Calculate Fourier Series approximation using formula
    fx.append(result.real)
fig=go.Figure()
plot(fig,x,box(x),'')
plot(fig,x,fx,'')
fig
'''

" TOO SLOW for COLAB\ndef calculateCn(M1,D,F):\n    for p in range(M1-1):\n        for q in range(M1-1):\n            D[p][q]=(e(n[p],xi[q],False))\n    cn=np.dot(D,F)/M1\n    return cn\n  ##Fourier Series Representation\n\n  M1=200 #number of grid points\n  M2 = 1500\n  h=np.linspace(0,M1-1,M1-1) #\n  xi=-np.pi+(2*np.pi/M1)*h\n  D=np.zeros((M1-1,M1-1),dtype=complex)\n  n=np.linspace(-M1/2,M1/2,M1-1)\n  F=box(xi,a[i],b[i])\n  cn=calculateCn(M1,D,F)\n  h2=np.linspace(0,M2-1,M2-1)\n  xi2 =-np.pi+ (2*np.pi/M2)*h2\n  xi=np.linspace(-np.pi,np.pi,100)\n  D2=np.zeros((M2-1,M1-1),dtype=complex)\n  for p in range(M2-1):\n    for q in range(M1-1):\n      D2[p][q]=(e(xi2[p],n[q]))\n\n  fx2 = (np.dot(D2,cn)).real\n  FourierBox.append(fx2)\n\n#TODO\n#figure out fft\n\n\nfx=[]\nfor xi in x:\n    result=0\n    for n in range(int((-N/2)),int((N/2))):\n      result=result+(cn[int(i+(N/2))]*(np.e**(1j*n*xi))) #Calculate Fourier Series approximation using formula\n    fx.append(result.real)\nfig=go.Figur

In [40]:

def fft(N):
    #w=np.e**((2*np.pi*-1j)/N)
    M = 40
    power=np.zeros((M,N),dtype=complex)
    xj = np.linspace(-np.pi, np.pi, N)
    for i in range(M):
        for j in range(N):
            #power[i][j]=((-N/2)+i)*j
            power[i][j] = -1j*(i-M/2)*xj[j]
    #DFT=(w**power)/N
    DFT = np.e**power
    return DFT


In [44]:

N=100
def f(x):
    return box(x)
x=np.linspace(-np.pi,np.pi,N)
fx=[]
cn=((np.dot(fft(N),f(x)))/N)
print(np.shape(cn))
M =40
for xi in x:
    result=0
    for n in range(int((-M/2)),(M-1)-int((M/2))):
      result=result+(cn[int(n+(M/2))]*(np.e**(1j*n*xi))) #Calculate Fourier Series approximation using formula
    fx.append(result.real)
fig=go.Figure()
plot(fig,x,fx,'')
plot(fig,x,f(x),'original')
fig


(40,)


In [65]:

N=100
M = 40
x=np.linspace(-np.pi,np.pi,N)

## Generating 1000 functions of each for testing model
#Box
a=np.linspace(np.pi/4,np.pi/2,1000)
b=np.append(np.linspace(-1,-0.01,500),np.linspace(0.01,1,500))
a=np.random.permutation(a)
b=np.random.permutation(b)
BoxFourier=[]
for i in range(1):
  '''
    F=box(x,a[i],b[i])
    fx=[]
    cn=((np.dot(fft(N),F))/N)
    print(np.shape(cn))
    for xi in x:
      result=0
      for n in range(int((-M/2)),(M-1)-int((M/2))):
        result=result+(cn[int(n+(M/2))]*(np.e**(1j*n*xi))) #Calculate Fourier Series approximation using formula
        fx.append(result.real)
    fx=np.array(fx)
    #normalization=np.max(abs(fx))
    BoxFourier.append(fx)#/normalization
    '''
  x=np.linspace(-np.pi,np.pi,N)
  fx=[]
  cn=((np.dot(fft(N),box(x,a[i],b[i])))/N)
  print(np.shape(cn))
  M =40
  for xi in x:
      result=0
      for n in range(int((-M/2)),(M-1)-int((M/2))):
        result=result+(cn[int(n+(M/2))]*(np.e**(1j*n*xi))) #Calculate Fourier Series approximation using formula
      fx.append(result.real)
print(np.shape(fx))
fig=go.Figure()
plot(fig,x,fx,'')
#plot(fig,x,f(x),'original')
fig



(40,)
(100,)


In [71]:

N=100
M = 40
x=np.linspace(-np.pi,np.pi,N)

## Generating 1000 functions of each for testing model
#Box
a=np.linspace(np.pi/4,np.pi/2,1000)
b=np.append(np.linspace(-1,-0.01,500),np.linspace(0.01,1,500))
a=np.random.permutation(a)
b=np.random.permutation(b)
BoxFourier=[]
for i in range(2):
    F=box(x,a[i],b[i])
    fx=[]
    cn=((np.dot(fft(N),F))/N)
    print(np.shape(cn))
    for xi in x:
      result=0
      for n in range(int((-M/2)),(M-1)-int((M/2))):
        result=result+(cn[int(n+(M/2))]*(np.e**(1j*n*xi))) #Calculate Fourier Series approximation using formula
      fx.append(result.real)
    fx=np.array(fx)
    normalization=np.max(abs(fx))
    BoxFourier.append(fx/normalization)

BoxFourier=np.array(BoxFourier)
print(np.shape(BoxFourier))   
fig=go.Figure()
normalization=np.max(abs(box(x,a[1],b[1])))
plot(fig,x,box(x,a[1],b[1])/normalization,'orig')
plot(fig,x,BoxFourier[1],'fourier')
fig



(40,)
(40,)
(2, 100)


In [52]:
BoxFourier=np.array(BoxFourier)
np.shape(BoxFourier)

(0,)

-0.9543687374749499

#### with noise

In [ ]:
## Generating 1000 functions of each for testing model
#Box
a=np.linspace(np.pi/4,np.pi/2,1000)
b=np.append(np.linspace(-1,-0.01,500),np.linspace(0.01,1,500))
a=np.random.permutation(a)
b=np.random.permutation(b)
BoxNoise=[]
for i in range(1000):
    F=box(x,a[i],b[i],True,np.random.uniform(0.1, 0.4))
    normalization=np.max(abs(F))
    BoxNoise.append(F/normalization)



### Saw Function

In [ ]:
a=np.linspace(np.pi/4,np.pi/2,1000)
b=np.append(np.linspace(-1,-0.01,500),np.linspace(0.01,1,500))
a=np.random.permutation(a)
b=np.random.permutation(b)
SawSignal=[]
#normalization=np.max(abs(saw(x,max(a),max(b))))
for i in range(1000):
    F=saw(x,a[i],b[i])
    normalization=np.max(abs(F))
    SawSignal.append(F/normalization)

#### Fourier

In [ ]:

N=100
M = 40
x=np.linspace(-np.pi,np.pi,N)

## Generating 1000 functions of each for testing model
#Box
a=np.linspace(np.pi/4,np.pi/2,1000)
b=np.append(np.linspace(-1,-0.01,500),np.linspace(0.01,1,500))
a=np.random.permutation(a)
b=np.random.permutation(b)
SawFourier=[]
for i in range(1000):
    F=saw(x,a[i],b[i])
    fx=[]
    cn=((np.dot(fft(N),F))/N)
    for xi in x:
      result=0
      for n in range(int((-M/2)),int((M/2))):
        result=result+(cn[int(n+(M/2))]*(np.e**(1j*n*xi))) #Calculate Fourier Series approximation using formula
        fx.append(result.real)
    fx=np.array(fx)
    normalization=np.max(abs(fx))
    SawFourier.append(fx/normalization)



#### with noise

In [ ]:
a=np.linspace(np.pi/4,np.pi/2,1000)
b=np.append(np.linspace(-1,-0.01,500),np.linspace(0.01,1,500))
a=np.random.permutation(a)
b=np.random.permutation(b)
SawNoise=[]
#normalization=np.max(abs(saw(x,max(a),max(b))))
for i in range(1000):
    F=saw(x,a[i],b[i],True,np.random.uniform(0.1, 0.4))
    normalization=np.max(abs(F))
    SawNoise.append(F/normalization)

### Exponential

In [ ]:
a=np.linspace(np.pi/4,np.pi/2,1000)
b=np.append(np.linspace(-1,-0.01,500),np.linspace(0.01,1,500))
c=np.append(np.linspace(-3,-1.01,500),np.linspace(-1.01,1,500))
a=np.random.permutation(a)
b=np.random.permutation(b)
c=np.random.permutation(c)
ExpSignal=[]
for i in range(1000):
    F=exp(x,a[i],b[i],c[i])
    normalization=np.max(abs(F))
    ExpSignal.append(F/normalization)

#### Fourier

In [ ]:

N=100
M = 40
x=np.linspace(-np.pi,np.pi,N)

a=np.linspace(np.pi/4,np.pi/2,1000)
b=np.append(np.linspace(-1,-0.01,500),np.linspace(0.01,1,500))
c=np.append(np.linspace(-3,-1.01,500),np.linspace(-1.01,1,500))
a=np.random.permutation(a)
b=np.random.permutation(b)
c=np.random.permutation(c)
ExpFourier=[]
for i in range(1000):
    F=exp(x,a[i],b[i],c[i])
    fx=[]
    cn=((np.dot(fft(N),F))/N)
    for xi in x:
      result=0
      for n in range(int((-M/2)),int((M/2))):
        result=result+(cn[int(n+(M/2))]*(np.e**(1j*n*xi))) #Calculate Fourier Series approximation using formula
        fx.append(result.real)
    fx=np.array(fx)
    normalization=np.max(abs(fx))
    ExpFourier.append(fx/normalization)

#### with noise

In [ ]:
a=np.linspace(np.pi/4,np.pi/2,1000)
b=np.append(np.linspace(-1,-0.01,500),np.linspace(0.01,1,500))
c=np.append(np.linspace(-3,-1.01,500),np.linspace(-1.01,1,500))
a=np.random.permutation(a)
b=np.random.permutation(b)
c=np.random.permutation(c)
ExpNoise=[]
for i in range(1000):
    F=exp(x,a[i],b[i],True,np.random.uniform(0.1, 0.4))
    normalization=np.max(abs(F))
    ExpNoise.append(F/normalization)

### Sinusoid

In [ ]:
a=np.linspace(np.pi/4,np.pi/2,1000)
b=np.append(np.linspace(-1,-0.01,500),np.linspace(0.01,2*np.pi,500))
c=np.append(np.linspace(-1,-0.01,500),np.linspace(0.01,1,500))
a=np.random.permutation(a)
b=np.random.permutation(b)
c=np.random.permutation(c)
SinSignal=[]
for i in range(1000):
    F=sinu(x,a[i],b[i],c[i])
    normalization=np.max(abs(F))
    SinSignal.append(F/normalization)

#### Fourier

In [ ]:

N=100
M = 40
x=np.linspace(-np.pi,np.pi,N)

a=np.linspace(np.pi/4,np.pi/2,1000)
b=np.append(np.linspace(-1,-0.5,500),np.linspace(0.5,1,500))
c=np.append(np.linspace(-3,-1.01,500),np.linspace(-1.01,1,500))
a=np.random.permutation(a)
b=np.random.permutation(b)
c=np.random.permutation(c)
SinFourier=[]
for i in range(1000):
    F=sinu(x,a[i],b[i],c[i])
    fx=[]
    cn=((np.dot(fft(N),F))/N)
    for xi in x:
      result=0
      for n in range(int((-M/2)),int((M/2))):
        result=result+(cn[int(n+(M/2))]*(np.e**(1j*n*xi))) #Calculate Fourier Series approximation using formula
        fx.append(result.real)
    fx=np.array(fx)
    normalization=np.max(abs(fx))
    SinFourier.append(fx/normalization)

#### with noise

In [ ]:
a=np.linspace(np.pi/4,np.pi/2,1000)
b=np.append(np.linspace(-1,-0.01,500),np.linspace(0.01,2*np.pi,500))
c=np.append(np.linspace(-1,-0.01,500),np.linspace(0.01,1,500))
a=np.random.permutation(a)
b=np.random.permutation(b)
c=np.random.permutation(c)
SinNoise=[]
for i in range(1000):
    F=sinu(x,a[i],b[i],c[i],True,np.random.uniform(0.1, 0.4))
    normalization=np.max(abs(F))
    SinNoise.append(F/normalization)

### Gaussian

In [ ]:
a=np.linspace(np.pi/4,np.pi/2,1000)
b=np.linspace(1,10,1000)
a=np.random.permutation(a)
b=np.random.permutation(b)

GausSignal=[]
for i in range(1000):
    F=gaus(x,a[i],int(b[i]))
    normalization=np.max(abs(F))
    GausSignal.append(F/normalization)

#### Fourier

In [ ]:

N=100
M = 40
x=np.linspace(-np.pi,np.pi,N)

## Generating 1000 functions of each for testing model
#Box
a=np.linspace(np.pi/4,np.pi/2,1000)
b=np.append(np.linspace(-1,-0.01,500),np.linspace(0.01,1,500))
a=np.random.permutation(a)
b=np.random.permutation(b)
GausFourier=[]
for i in range(1000):
    F=gaus(x,a[i],int(b[i]))
    fx=[]
    cn=((np.dot(fft(N),F))/N)
    for xi in x:
      result=0
      for n in range(int((-M/2)),int((M/2))):
        result=result+(cn[int(n+(M/2))]*(np.e**(1j*n*xi))) #Calculate Fourier Series approximation using formula
        fx.append(result.real)
    fx=np.array(fx)
    normalization=np.max(abs(fx))
    GausFourier.append(fx/normalization)



#### with noise

In [ ]:
a=np.linspace(np.pi/4,np.pi/2,1000)
b=np.linspace(-10,10,1000)
a=np.random.permutation(a)
b=np.random.permutation(b)

GausNoise=[]
for i in range(1000):
    F=gaus(x,a[i],int(b[i]),True,np.random.uniform(0.1, 0.4))
    normalization=np.max(abs(F))
    GausNoise.append(F/normalization)

### Combining Results

In [ ]:
#Combining Signals
Signals=[BoxSignal,SawSignal,ExpSignal,SinSignal,GausSignal]
SignalsWNoise=[[BoxNoise,SawNoise,ExpNoise,SinNoise,GausNoise]]
SignalsFourier=[[BoxFourier,SawFourier,ExpFourier,SinFourier,GausFourier]]
SignalLabels=[
    np.zeros(1000),
    1*np.ones(1000),
    2*np.ones(1000),
    3*np.ones(1000),
    4*np.ones(1000)]
SignalsTest=np.array(np.concatenate(Signals).flat)
SignalsTest=SignalsTest.reshape((5000,100))
SignalsWNoiseTest=np.array(np.concatenate(SignalsWNoise).flat)
SignalsWNoiseTest=SignalsWNoiseTest.reshape((5000,100))
SignalsFourierTest=np.array(np.concatenate(SignalsFourier).flat)
#SignalsFourierTest=SignalsFourierTest.reshape((5000,100))
SignalLabelsTest=np.array(np.concatenate(SignalLabels).flat)

## Visualization of Data of N samples from test Data

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

N=5
fig =make_subplots(
    rows=N,cols=5,
    subplot_titles=("Box Signal","Saw Signal", "Exponential Signal","Sinusoid Signal","Gaussian Signal")
    )
fig.update_xaxes(range=[-np.pi,np.pi])
#fig.update_yaxes(range=[-1,1])

sample=np.random.choice(np.linspace(0,1000,1000), N, replace=False)
sample=np.around(sample)
for i in range(N):
    index=int(sample[i])
    fig.add_trace(
        go.Scatter(
            x=x,y=BoxSignal[index],name='Constant Signal %d'%i,mode="lines"
        ),
        row=i+1,col=1
    )
for i in range(N):
    index=int(sample[i])
    fig.add_trace(
        go.Scatter(
            x=x,y=SawSignal[index],name='Line Signal %d'%i,mode="lines"
        ),
        row=i+1,col=2
    )
for i in range(N):
    index=int(sample[i])
    fig.add_trace(
        go.Scatter(
            x=x,y=ExpSignal[index],name='Exp Signal %d'%i,mode="lines"
        ),
        row=i+1,col=3
    )
for i in range(N):
    index=int(sample[i])
    fig.add_trace(
        go.Scatter(
            x=x,y=SinSignal[index],name='Sinusoid Signal %d'%i,mode="lines"
        ),
        row=i+1,col=4
    )
for i in range(N):
    index=int(sample[i])
    fig.add_trace(
        go.Scatter(
            x=x,y=GausSignal[index],name='Gaus Signal %d'%i,mode="lines"
        ),
        row=i+1,col=5
    )
#fig.update_xaxes(matches='x')
fig.update_layout(height=1200,width=1000,showlegend=False, title_text="%d Test Samples of Generated Signals w/o noise"%N)
fig.show()



### Fourier Data

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

N=10
fig =make_subplots(
    rows=N,cols=5,
    subplot_titles=("Box Signal","Saw Signal", "Exponential Signal","Sinusoid Signal","Gaussian Signal")
    )
fig.update_xaxes(range=[-np.pi,np.pi])
#fig.update_yaxes(range=[-1,1])

sample=np.random.choice(np.linspace(0,1000,1000), N, replace=False)
sample=np.around(sample)
for i in range(N):
    index=int(sample[i])
    fig.add_trace(
        go.Scatter(
            x=x,y=BoxFourier[index],name='Constant Signal %d'%i,mode="lines"
        ),
        row=i+1,col=1
    )
for i in range(N):
    index=int(sample[i])
    fig.add_trace(
        go.Scatter(
            x=x,y=SawFourier[index],name='Line Signal %d'%i,mode="lines"
        ),
        row=i+1,col=2
    )
for i in range(N):
    index=int(sample[i])
    fig.add_trace(
        go.Scatter(
            x=x,y=ExpFourier[index],name='Exp Signal %d'%i,mode="lines"
        ),
        row=i+1,col=3
    )
for i in range(N):
    index=int(sample[i])
    fig.add_trace(
        go.Scatter(
            x=x,y=SinFourier[index],name='Sinusoid Signal %d'%i,mode="lines"
        ),
        row=i+1,col=4
    )
for i in range(N):
    index=int(sample[i])
    fig.add_trace(
        go.Scatter(
            x=x,y=GausFourier[index],name='Gaus Signal %d'%i,mode="lines"
        ),
        row=i+1,col=5
    )
#fig.update_xaxes(matches='x')
fig.update_layout(height=1200,width=1000,showlegend=False, title_text="%d Test Samples of Fourier Data"%N)
fig.show()



Output hidden; open in https://colab.research.google.com to view.

### With Noise

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

N=5
fig =make_subplots(
    rows=N,cols=5,
    subplot_titles=("Box Signal","Saw Signal", "Exponential Signal","Sinusoid Signal","Gaussian Signal")
    )
fig.update_xaxes(range=[-np.pi,np.pi])
#fig.update_yaxes(range=[-1,1])

sample=np.random.choice(np.linspace(0,1000,1000), N, replace=False)
sample=np.around(sample)
for i in range(N):
    index=int(sample[i])
    fig.add_trace(
        go.Scatter(
            x=x,y=BoxNoise[index],name='Constant Signal %d'%i,mode="lines"
        ),
        row=i+1,col=1
    )
for i in range(N):
    index=int(sample[i])
    fig.add_trace(
        go.Scatter(
            x=x,y=SawNoise[index],name='Line Signal %d'%i,mode="lines"
        ),
        row=i+1,col=2
    )
for i in range(N):
    index=int(sample[i])
    fig.add_trace(
        go.Scatter(
            x=x,y=ExpNoise[index],name='Exp Signal %d'%i,mode="lines"
        ),
        row=i+1,col=3
    )
for i in range(N):
    index=int(sample[i])
    fig.add_trace(
        go.Scatter(
            x=x,y=SinNoise[index],name='Sinusoid Signal %d'%i,mode="lines"
        ),
        row=i+1,col=4
    )
for i in range(N):
    index=int(sample[i])
    fig.add_trace(
        go.Scatter(
            x=x,y=GausNoise[index],name='Gaus Signal %d'%i,mode="lines"
        ),
        row=i+1,col=5
    )
#fig.update_xaxes(matches='x')
fig.update_layout(height=1200,width=1000,showlegend=False, title_text="%d Test Samples of Generated Signals with Noise"%N)
fig.show()




## Training Model 

### Neural Network(15 epochs, 128 RELU activation layer)

In [ ]:
modelNN = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(5)
])
modelNN.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
modelNN.fit(SignalsTrain, SignalLabelsTrain, epochs=100)


Epoch 1/100
1563/1563 [==============================] - 5s 3ms/step - loss: 0.4410 - accuracy: 0.7990
Epoch 2/100
1563/1563 [==============================] - 4s 2ms/step - loss: 0.2745 - accuracy: 0.8702
Epoch 3/100
1563/1563 [==============================] - 4s 3ms/step - loss: 0.2373 - accuracy: 0.8906
Epoch 4/100
1563/1563 [==============================] - 5s 3ms/step - loss: 0.2196 - accuracy: 0.9010
Epoch 5/100
1563/1563 [==============================] - 4s 2ms/step - loss: 0.2072 - accuracy: 0.9099
Epoch 6/100
1563/1563 [==============================] - 4s 2ms/step - loss: 0.1975 - accuracy: 0.9146
Epoch 7/100
1563/1563 [==============================] - 4s 2ms/step - loss: 0.1894 - accuracy: 0.9191
Epoch 8/100
1563/1563 [==============================] - 4s 2ms/step - loss: 0.1831 - accuracy: 0.9231
Epoch 9/100
1563/1563 [==============================] - 4s 2ms/step - loss: 0.1796 - accuracy: 0.9244
Epoch 10/100
1563/1563 [==============================] - 4s 3ms/step - l

### Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
modelDT = DecisionTreeClassifier(max_depth=100, random_state=1111)
modelDT.fit(SignalsTrain, SignalLabelsTrain)

DecisionTreeClassifier(max_depth=100, random_state=1111)

## Testing Model


In [ ]:
SignalsFourierTest.shape

(5000, 100)

In [ ]:
SignalsFourierTest=SignalsFourierTest.reshape(5000,4000)

In [ ]:
SignalsFourierTest=np.resize(SignalsFourierTest,(5000,100))

### NN Model

In [ ]:
test_loss, woNoiseAcc = modelNN.evaluate(SignalsTest,  SignalLabelsTest, verbose=2)
test_loss, wnoiseAcc = modelNN.evaluate(SignalsWNoiseTest,  SignalLabelsTest, verbose=2)
test_loss, wfourier = modelNN.evaluate(SignalsFourierTest,  SignalLabelsTest, verbose=2)
print("Signals with out Noise: %.5f \n Signals with Noise: %.5f\n Signals Fourier: %.5f"%(woNoiseAcc,wnoiseAcc,wfourier))


157/157 - 0s - loss: 0.1132 - accuracy: 0.9584 - 217ms/epoch - 1ms/step
157/157 - 0s - loss: 17.0540 - accuracy: 0.4598 - 209ms/epoch - 1ms/step
157/157 - 0s - loss: 62.4272 - accuracy: 0.2010 - 187ms/epoch - 1ms/step
Signals with out Noise: 0.95840 
 Signals with Noise: 0.45980
 Signals Fourier: 0.20100


#### DT Model

In [ ]:
a=np.array(modelDT.predict(SignalsWNoiseTest))
b=np.array(SignalLabelsTest)
(a==b).sum()/len(SignalsWNoiseTest)

0.2742